In [1]:
from netCDF4 import Dataset
from fvcom import FvcomGrid
import numpy as np

Get some FVCOM output to examine.

In [2]:
ds = Dataset('/net/babaracus/home/benr/wqmodels/ssm/unionriver/hyd/1x_1pass/OUTPUT/netcdf/ssm_00001.nc')
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: FVCOM Velocity Blockage Test(Updated Block with Kelp June 2013)                 
    institution: School for Marine Science and Technology
    source: FVCOM_2.7
    history: model started at: 17/08/2022   07:51
    references: http://fvcom.smast.umassd.edu
    Conventions: CF-1.0
    dimensions(sizes): scalar(1), node(16012), nele(25019), siglay(10), siglev(11), three(3), four(4), obc(87), obc2(87), time(24)
    variables(dimensions): int32 nprocs(scalar), int32 partition(nele), float32 Initial_Density(siglay, node), float32 x(node), float32 y(node), float32 lon(node), float32 lat(node), float32 siglay(siglay), float32 siglay_shift(siglay), float32 siglev(siglev), float32 h(node), float32 nv(three, nele), float32 a1u(four, nele), float32 a2u(four, nele), float32 aw0(three, nele), float32 awx(three, nele), float32 awy(three, nele), float32 time(time), int32 iint(time), float32 u(t

Define the important parts of a section needed for this proof-of-concept. We need to know an element, plus the previous and next adjacent elements in the section.

This code does not handle edge elements properly.

In [3]:
prev_ele = 7823
ele = 7652
next_ele = 7653

In [4]:
grid = FvcomGrid.from_output(ds, calc=True)
neis = (grid.nbe[:,ele-1]-1).T
neis = neis[~(neis == -1)]
which_prev_nei = (grid.nbe[:, ele-1] == prev_ele).nonzero()[0]
non_nei_idxs = (np.arange(3) != which_prev_nei).nonzero()[0]
common_nodes = grid.nv[non_nei_idxs, ele-1]
xm1, ym1 = grid.ncoord[0:2, common_nodes-1].mean(axis=1)

which_next_nei = (grid.nbe[:, ele-1] == next_ele).nonzero()[0]
non_nei_idxs = (np.arange(3) != which_next_nei).nonzero()[0]
common_nodes = grid.nv[non_nei_idxs, ele-1]
xm2, ym2 = grid.ncoord[0:2, common_nodes-1].mean(axis=1)
xm1, ym1, xm2, ym2

(516422.8, 5332524.5, 516387.94, 5333294.0)

In [5]:
x, y = grid.elcoord[0:2, ele-1]
dx1 = xm1 - x
dy1 = ym1 - y
th1 = np.arctan2(-dy1, -dx1)
th1 = np.where(th1 < 0, th1 + 2 * np.pi, th1) - np.pi / 2 % (2 * np.pi)
dx2 = xm2 - x
dy2 = ym2 - y
th2 = np.arctan2(dy2, dx2)
th2 = np.where(th2 < 0, th2 + 2 * np.pi, th2) - np.pi / 2 % (2 * np.pi)
n1 = np.array([np.cos(th1), np.sin(th1)])
n2 = np.array([np.cos(th2), np.sin(th2)])

In [6]:
us = ds['u'][:,:,neis]
vs = ds['v'][:,:,neis]
du = (us.T - ds['u'][:,:,ele-1].T).T
dv = (vs.T - ds['v'][:,:,ele-1].T).T
dxy = (grid.elcoord[0:2,neis].T - grid.elcoord[0:2,ele-1].T)
du[20,2], dv[20,2], dxy

(masked_array(data=[-0.31615293,  0.04517227,  0.05778968],
              mask=False,
        fill_value=1e+20,
             dtype=float32),
 masked_array(data=[ 0.39888147, -0.18213874, -0.06463821],
              mask=False,
        fill_value=1e+20,
             dtype=float32),
 array([[ 326.59375,  677.5    ],
        [ 444.34375, -473.5    ],
        [-737.21875,   11.     ]], dtype=float32))

This is the manual way: compute the least-squares coefficients for each velocity component, at every sigma layer, at every timestep. This is $O(2 K_B N)$. This is going to be sloooow

In [7]:
au, bu = np.linalg.lstsq(dxy, du[20,2], rcond=None)[0]
av, bv = np.linalg.lstsq(dxy, dv[20,2], rcond=None)[0]
au, bu

(-0.00014728414, -0.0003432317)

The more efficient method is to compute the pseudo-inverse of dxy, which can then be multiplied with each velocity difference to get coefficients

In [8]:
dxy_pinv = np.linalg.pinv(dxy)
dxy_pinv

array([[ 3.8209284e-04,  5.2650401e-04, -8.6983963e-04],
       [ 9.8992500e-04, -6.9505814e-04,  1.9612371e-05]], dtype=float32)

In [9]:
dxy_pinv @ du[20,2]

masked_array(data=[-0.00014728, -0.00034323],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

The uniform velocity model (what Conroy's code does).

Assume the velocity at the centroid is a good estimate of the velocity throughout the element. Then, to get the transport flux, just dot with the normal to each segment (side midpoint to centroid) and scale by the segment length over the total length.

In [10]:
u0 = ds['u'][:,:,ele-1]
v0 = ds['v'][:,:,ele-1]
l1 = np.sqrt(dx1 ** 2 + dy1 ** 2)
l2 = np.sqrt(dx2 ** 2 + dy2 ** 2)
l = l1 + l2
tf_uniform = l1/l * (u0[20,2] * n1[0] + v0[20,2] * n1[1]) + l2/l * (u0[20,2] * n2[0] + v0[20,2] * n2[1])
tf_uniform

-0.7855390714137171

The least-squares velocity model. This is derived from the FVCOM manual to get velocity through the same segments that Conroy uses, but without assuming uniform velocity.

The transport through the element per unit length of segment is a line integral:

$TF*l = \int \vec{u} \cdot \vec{n} ds$

Decompose into each segment and parameterize the line integral, treating the centroid as the origin of the coordinate system and the side midpoints as $(x_1, y_1)$ and $(x_2, y_2)$:

$y = y_1 t$ (or $y_2 t$)

$x = x_1 t$ (or $x_2 t$)

$TF*l = \int_{t=0}^1 ((un_{x,1} + vn_{y,1}) \sqrt{x_1^2 + y_1^2} + (un_{x,2} + vn_{y,2})\sqrt{x_2^2 + y_2^2})dt$

Let $l_j = \sqrt{x_j^2 + y_j^2}$. Equations for $u$ and $v$ are in the FVCOM manual (3.20 and 3.21). Substitute:

$TF*l = \int_{t=0}^1 (n_{x,1}(u_0 + a^u x_1 t + b^u y_1 t) + n_{y,1}(v_0 + a^v x_1 t + b^v y_1 t))l_1 + (n_{x,2}(u_0 + a^u x_2 t + b^u y_2 t) + n_{y,2}(u_0 + a^v x_2 t + b^v y_2 t))l_2)dt$

Integrating gives the following result:

$TF = \frac{l_1}{l} (n_{x,1} (u_0 + 0.5 (a^u x_1 + b^u y_1)) + n_{y,1}(v_0 + 0.5 (a^v x_1 + b^v y_1))) + \frac{l_2}{l} (n_{x,2} (u_0 + 0.5 (a^u x_2 + b^u y_2)) + n_{y,2}(v_0 + 0.5 (a^v x_2 + b^v y_2)))$

In [11]:
au, bu = dxy_pinv @ du[20,2]
av, bv = dxy_pinv @ dv[20,2]
tf_lstsq = (l1 * (n1[0] * (u0[20,2] + 0.5 * (au * dx1 + bu * dy1))
        + n1[1] * (v0[20,2] + 0.5 * (av * dx1 + bv * dy1))
      ) + l2 * (n2[0] * (u0[20,2] + 0.5 * (au * dx2 + bu * dy2))
        + n2[1] * (v0[20,2] + 0.5 * (av * dx2 + bv * dy2))
    )) / l

tf_lstsq

-0.8545657243944654

The above procedure works well, but it's not implemented efficiently since it depends on performing $O(K_B N)$ matrix multiplications to get all of the $a$ and $b$ constants. A more efficient method is to reshape $du$ and $dv$ into matrices that have one row per element neighbor, and the columns contain flattened values of $u$ and $v$ at different sigma levels and times. Multiplying the pseudo-inverse with each reshaped matrix will give matrices containing all of the constants $a^u, b^u. a^v, b^v$, which must be reshaped back. Then the transport function can be computed in a single line.

In [12]:
du_reshape = du.T.reshape(du.shape[2], (du.shape[0]*du.shape[1])).data
dv_reshape = dv.T.reshape(dv.shape[2], (dv.shape[0]*dv.shape[1])).data
assert (du[20,2] == du_reshape[:,2 * du.shape[0] + 20]).all()
abu_reshape = dxy_pinv @ du_reshape
abv_reshape = dxy_pinv @ dv_reshape
assert (np.abs(np.array([au, bu]) - abu_reshape[:,2 * du.shape[0] + 20]) < 0.000001).all()

abu = abu_reshape.reshape(2, du.shape[1], du.shape[0]).T
abv = abv_reshape.reshape(2, dv.shape[1], dv.shape[0]).T
assert (np.abs(np.array([au, bu]) - abu[20,2]) < 0.000001).all()

tf_lstsq_noloop = (l1 * (n1[0] * (u0 + 0.5 * (abu[:,:,0] * dx1 + abu[:,:,1] * dy1))
                         + n1[1] * (v0 + 0.5 * (abv[:,:,0] * dx1 + abv[:,:,1] * dy1))
                        ) + l2 * (n2[0] * (u0 + 0.5 * (abu[:,:,0] * dx2 + abu[:,:,1] * dy2))
                                  + n2[1] * (v0 + 0.5 * (abv[:,:,0] * dx2 + abv[:,:,1] * dy2))
                                 )) / l
assert tf_lstsq - tf_lstsq_noloop[20,2] < 0.000001